In [1]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

In [ ]:
#Script to write csv to use in Deep Learning 

In [3]:
#open csv with informations about dataset
csv_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] 

In [5]:
#liste of generated nifti by "nifti_builder_from_json" script 

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'
liste_nifti = os.listdir(nifti_directory)

In [6]:
liste = []
type_ = 'pet0'
for row in csv_data : 
    subliste = []
    subliste.append(row[0]) #patient id 
    #add other informations if need 

    study_uid = row[2]
    subliste.append(study_uid)

    subsubliste = []
    for nifti in liste_nifti : 
        if study_uid in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : #or == 3 if there is a mask 
        subsubliste = sorted(subsubliste)
        subliste.append(subsubliste[0])
        subliste.append(subsubliste[1])
        #subliste.append(subsubliste[2]) #add it if there is a mask 
    else : 
        pass
    subliste.append(type_)
    liste.append(subliste)



In [8]:
liste[0]

['11011101021011',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_CT.nii',
 '1.2.250.1.59.470.940100027.1.20140619143155.5902.130185_nifti_PT.nii',
 'pet4']

In [9]:
#clean serie : must have patient ID, study_UID, CT_path, PET_path, MASK_path if pet0, and type "PET" (0,2,4)
completed_liste = []
for serie in liste : 
    if len(serie) == 5 : #== 6 if Mask_path 
        completed_liste.append(serie)



In [11]:
#here check for double : if there are 2 patient with same ID, or same STUDY_UID, delete the other(s) serie(s)

dataset = []
dataset.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in dataset : 
        dataset.append(completed_liste[i])

In [24]:
dataset[0]

['11011101021014',
 '1.2.840.113704.1.111.3608.1434978016.23',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_CT.nii',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_PT.nii',
 'pet2']

In [19]:
#create folder to save nifti

for serie in dataset: 
    patient_id = serie[0]
    pet_type = serie[-1]
    nifti_ct = serie[2]
    nifti_pet = serie[3]
    #nifti_mask = serie[5] #if pet0

    folder = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "Déjà creer => already folder for ", pet_type)

    ct_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_ct
    pt_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET4'+'/'+nifti_pet
        #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
    #new_mask_path = subfolder+'/'+nifti_mask
    try : 

        os.rename(ct_path, subfolder+'/'+nifti_ct)
        os.rename(pt_path, subfolder+'/'+nifti_pet)
        #os.rename(mask_path, subfolder+'/'+nifti_mask)


        serie[2] = new_ct_path
        serie[3] = new_pet_path 
        serie[4] = new_mask_path
    except Exception as err : 
        print(err)


8024b95f92/GAINED_NIFTI/11011108371001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371001/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371002/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371003/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108371004/pet4 created
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/11011108611001/pet4 created
/

In [38]:
#write csv , add NIFTI_MASK column if there is a mask 
nifti_directory = 
filename = ''
with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET"])
    for serie in new_liste: 
        csv_writer.writerow([serie[0], serie[1], serie[-1], serie[2], serie[3]])